MACHINE LEARNING APPROACH

This challenge is not well suited for calassic machine learning approach

The larged set of labeld examples for training is the training set with 96 labeled examples
- 38 REFUTES
- 18 SUPPORTS
- 40 NOT ENOUGH INFO
The dataset is not balanced

The test set is not labeled so can't be use for training or testing (results can't be verified)

The dev set contains 32 labeled rumor so can be used for testing

Can't supplement the traing set with dev set examples due to overfitting problems

Training set opening

In [1]:
import json

with open('../Datasets/English_train_v3.json', 'r') as f:
    trainSet = json.load(f)

Development set opening

In [2]:
with open('../Datasets/English_dev.json', 'r') as f:
    devSet = json.load(f)

VECTORIZATION

Training e "Test" sets have to be vectorized together to have the same number of features

In [3]:
#Training rumor list
trainingList = []
for rumor in trainSet:
    trainingList.append(rumor["rumor"])

In [4]:
dimTrain = len(trainingList)

In [5]:
#Test rumor list
testList = []
for rumor in devSet:
    testList.append(rumor["rumor"])

In [6]:
dimTest = len(testList)

Creating a single list to vectorize

In [7]:
listToVectorize = trainingList + testList

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine

def vectorizer(rumors):
    
    vectorizer = TfidfVectorizer(use_idf=True,sublinear_tf=True,smooth_idf=True, stop_words=None)
    tfidf_matrix = vectorizer.fit_transform(rumors)

    return tfidf_matrix

In [9]:
vectorizedList = vectorizer(listToVectorize)

Divide the vectorization in training and test

In [10]:
trainVett = vectorizedList[:-dimTest]
testVett = vectorizedList[-dimTest:]

In [11]:
X = trainVett

#Recovering the labels
Y = []
for rumor in trainSet:
    Y.append(rumor["label"])

CLASSIFICATION

The classification is done using Naive Bayes due to the scarsity of training examples

In [12]:
from sklearn.naive_bayes import MultinomialNB

NB = MultinomialNB()

model= NB.fit(X,Y)

In [13]:
predictionList = model.predict(testVett)

Label extraction from the test

In [14]:
true_labels = []
for element in devSet:
    true_labels.append(element['label'])

Scoring the Macro-F1 to have an idea ot the performance

In [15]:
from sklearn.metrics import f1_score

f1_score(true_labels, predictionList, labels=["SUPPORTS", "REFUTES", "NOT ENOUGH INFO"],average='macro')

0.2574074074074074

In [16]:
def flat_list(lista):
    ris = []
    for element in lista.toarray():
        ris.extend(element)
    return ris

SAVING RESULTS

Saving the classification results using the format given by the challenge

In [17]:
from scipy.spatial.distance import cosine
import numpy

dictionaryList = []
listToVectorize = []

for rumor,prediction in zip(devSet,predictionList):
    #Rumor and timeline in the same vector
    listToVectorize.append(rumor['rumor'])
    for evidence in rumor['timeline']:
        listToVectorize.append(evidence[2])
        
    #Vectorization
    vectorizedList = vectorizer(listToVectorize)

    #Adding the cosine similarity to the timeline
    vettRumor = vectorizedList[0]
    vectorizedEvidence = vectorizedList[1:]
    evidenceList = rumor['timeline']
    i = 0

    for evidence,vettEvidence in zip(evidenceList,vectorizedEvidence):
        evidence.append(cosine(numpy.transpose(flat_list(vettRumor)),numpy.transpose(flat_list(vettEvidence))))

    #Inverse sorting, the lower the better
    evidenceList.sort(key=lambda x: x[3])

    #Taking the five best scoring
    evidenceList = evidenceList[:5]

    dictionary = {
            "id":rumor['id'],
            "predicted_label":prediction,
            "predicted_evidence":evidenceList
        }
    dictionaryList.append(dictionary)

In [19]:
with open("../Results/Verification/NaiveBayes_verification_results.json", "w") as fr:
    json.dump(dictionaryList, fr)

Saving the evidence retrieval results using the format given by the challenge

In [20]:
with open("../Results/Evidence/NaiveBayes_evidence_results.txt", "w") as fe:
    for rumor in dictionaryList:
        i = 1
        for evid in rumor['predicted_evidence']:
            fe.write(rumor['id']+"\tQ0\t"+evid[1]+"\t"+str(i)+"\t"+str(evid[3])+"\tIO\n")
            i = i+1